In [2]:
import osmnx as ox
import random
from shapely.geometry import Point
import geopandas as gpd
import folium

# 1. Download da malha viária de Belém
print("Baixando ruas de Belém...")


#network_type	O que inclui
#"drive"	Apenas vias em que carros podem trafegar (ruas, avenidas, rodovias). Exclui ciclovias e caminhos de pedestre.
#"drive_service"	Como “drive”, mas inclui ruas de serviço (acessos, estacionamentos, etc.).
#"walk"	Todas vias caminháveis (calçadas, trilhas urbanas, passagens).
#"bike"	Vias cicláveis.
#"all"	Todas as vias possíveis.
#"all_private"	Tudo, incluindo vias privadas.
G = ox.graph_from_place("Belém, Pará, Brazil", network_type="drive")

# 2. Converter grafo para GeoDataFrame contendo as ruas
edges = ox.graph_to_gdfs(G, nodes=False, edges=True)



Baixando ruas de Belém...


In [3]:
# 3. Função para gerar pontos aleatórios ao longo das ruas
def gerar_ponto_aleatorio_em_rua(geom):
    distancia_random = random.uniform(0, geom.length)
    ponto = geom.interpolate(distancia_random)
    return ponto

# 4. Gerar os 50 pontos
pontos = []
for _ in range(70):
    rua_escolhida = edges.sample(1).iloc[0]
    geom = rua_escolhida.geometry
    ponto = gerar_ponto_aleatorio_em_rua(geom)
    pontos.append(ponto)

# 5. Criar GeoDataFrame dos pontos
gdf_pontos = gpd.GeoDataFrame(geometry=pontos, crs="EPSG:4326")
gdf_pontos["lat"] = gdf_pontos.geometry.y
gdf_pontos["lon"] = gdf_pontos.geometry.x

# Salvar pontos
gdf_pontos[["lat", "lon"]].to_csv("pontos_irregulares_belem.csv", index=False)
print("\nArquivo 'pontos_irregulares_belem.csv' salvo com sucesso!")



Arquivo 'pontos_irregulares_belem.csv' salvo com sucesso!


In [4]:
import pandas as pd
# import folium

# 1. Ler o CSV salvo anteriormente
descarte = pd.read_csv("pontos_irregulares_belem.csv")

# 2. Calcular o centro do mapa
centro = [descarte["lat"].mean(), descarte["lon"].mean()]

# 3. Criar o mapa
mapa = folium.Map(location=centro, zoom_start=12)

# 4. Adicionar os pontos ao mapa
for idx, row in descarte.iterrows():
    folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=4,
        color="red",
        fill=True,
        fill_color="red",
    ).add_to(mapa)

# 5. Exibir o mapa
mapa


In [6]:
#gerando pontos aleatorios em bairros especificos 
import os

filename="shape_bairros.gpkg"
path='/home/akel/PycharmProjects/Data_ambiental/data/process/'
arquivo = os.path.join(path, filename)

gdf_bairros = gpd.read_file(arquivo)

bairros4674 = gdf_bairros.to_crs("EPSG:4674")
edges = edges.to_crs("EPSG:4674")

In [7]:
bairro_escolhido = bairros4674[bairros4674["NM_BAIRRO"] == "Condor"]

ruas_no_bairro = gpd.overlay(
    edges,
    bairro_escolhido,
    how="intersection"
)

pontos = []
for _ in range(7):
    rua_escolhida = ruas_no_bairro.sample(1).iloc[0]
    geom = rua_escolhida.geometry
    ponto = gerar_ponto_aleatorio_em_rua(geom)
    pontos.append(ponto)

# 5. Criar GeoDataFrame dos pontos
gdf_pontos = gpd.GeoDataFrame(geometry=pontos, crs="EPSG:4326")
gdf_pontos["lat"] = gdf_pontos.geometry.y
gdf_pontos["lon"] = gdf_pontos.geometry.x

In [8]:
# 2. Calcular o centro do mapa
centro = [gdf_pontos["lat"].mean(), gdf_pontos["lon"].mean()]

# 3. Criar o mapa
mapa = folium.Map(location=centro, zoom_start=12)

# 4. Adicionar os pontos ao mapa
for idx, row in gdf_pontos.iterrows():
    folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=4,
        color="red",
        fill=True,
        fill_color="red",
    ).add_to(mapa)

# 5. Exibir o mapa
mapa